In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip -q install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == 

In [4]:
import torch
import gc
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from datasets import load_dataset
import evaluate

2025-07-09 15:30:27.790972: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752075028.153001      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752075028.262295      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [58]:
# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


# Task 1: Data Preprocessing

In [5]:
# TODO: Load tokenizer and set pad_token
tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
# Set pad_token (GPT-2 has no pad_token by default, so we use eos_token as pad_token)
tokenizer.pad_token = tokenizer.eos_token

In [7]:
# TODO: Load model configuration and model
config = AutoConfig.from_pretrained("gpt2")  # Load GPT-2 model configuration
model = AutoModelForCausalLM.from_pretrained("gpt2", config=config) # Load GPT-2 causal language model with the configuration

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
from sklearn.model_selection import train_test_split
import random
import gc

In [9]:
# TODO: Load SQuAD dataset and create train/validation splits
dataset = load_dataset("squad")
vali_ds = vali_ds = dataset['validation'].select(range(5)) #select 5 examples for validation
split_ds = split_ds = dataset['train'].train_test_split(test_size=0.1, seed=42) #train_test_split with test_size=0.1
train_ds = split_ds['train'].shuffle(seed=42).select(range(2000)) #select 2000 shuffled examples
eval_ds = split_ds['test'].shuffle(seed=42).select(range(200)) #select 200 shuffled examples

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [10]:
print("Train Dataset size:", len(train_ds))
print("Eval Dataset size:", len(eval_ds))
print("Validation Dataset size:", len(vali_ds))

Train Dataset size: 2000
Eval Dataset size: 200
Validation Dataset size: 5


In [11]:
print("Example from Train Dataset:", train_ds[0])
print("___________________________________________")
print("Example from Eval Dataset:", eval_ds[0])
print("___________________________________________")
print("Example from Validation Dataset:", vali_ds[0])

Example from Train Dataset: {'id': '5730eac6b7151e1900c015d4', 'title': 'Immaculate_Conception', 'context': 'On 28 February 1476, Pope Sixtus IV, authorized those dioceses that wished to introduce the feast to do so, and introduced it to his own diocese of Rome in 1477, with a specially composed Mass and Office of the feast. With his bull Cum praeexcelsa of 28 February 1477, in which he referred to the feast as that of the Conception of Mary, without using the word "Immaculate", he granted indulgences to those who would participate in the specially composed Mass or Office on the feast itself or during its octave, and he used the word "immaculate" of Mary, but applied instead the adjective "miraculous" to her conception. On 4 September 1483, referring to the feast as that of "the Conception of Immaculate Mary ever Virgin", he condemned both those who called it mortally sinful and heretical to hold that the "glorious and immaculate mother of God was conceived without the stain of origina

In [23]:
print(train_ds.column_names)
print(eval_ds.column_names)
print(vali_ds.column_names)

['id', 'title', 'context', 'question', 'answers']
['id', 'title', 'context', 'question', 'answers']
['id', 'title', 'context', 'question', 'answers']


#### Free up memory by deleting large unused datasets \
#### Force garbage collection to immediately release memory resources

In [12]:
# Clear memory
del dataset, split_ds
gc.collect()

220

###  Handle edge cases (empty answers, truncation)

In [13]:
def preprocess_function(examples):
    """
    Creates properly aligned input-label pairs for causal LM fine-tuning
    
    Args:
        examples: Batch of SQuAD examples with 'context', 'question', 'answers'
    
    Returns:
        Dictionary with 'input_ids', 'attention_mask', 'labels'
        
    TODO: Implement the following steps:
    1. For each example, create a prompt: "Context: {context}\nQuestion: {question}\nAnswer: "
    2. Concatenate prompt + answer + eos_token
    3. Tokenize the full sequence
    4. Create labels by copying input_ids
    5. Mask prompt tokens in labels with -100 (so loss only computed on answer)
    """
    inputs = [] #Prompt + Answer + eos
    #targets = []
    ### Build prompt + answer for each example
    for context, question, answers in zip(examples['context'], examples['question'], examples['answers']):
        # TODO: Extract answer text (use first answer if multiple)
        answer_text = answers['text'][0] if answers['text'] else ''
        
        # TODO: Create prompt and full sequence
        prompt = f"Context: {context}\nQuestion: {question}\nAnswer: "
        full_text = prompt + answer_text + tokenizer.eos_token
        
        inputs.append(full_text)
    
    # TODO: Tokenize all sequences ##### no padding here, only truncation
    model_inputs = tokenizer(inputs, padding=False, truncation=True, max_length=256, return_tensors=None)
    
    # TODO: Create labels with prompt masking
    labels = []
    for i, input_ids in enumerate(model_inputs['input_ids']):
        # YOUR CODE HERE: Create label_ids, find prompt_length, mask prompt tokens
                
        # Get prompt text again to find its tokenized length
        prompt_text = inputs[i].split("Answer:")[0] + "Answer: "
        
        # Tokenize prompt to get length
        prompt_ids = tokenizer(prompt_text, add_special_tokens=False)['input_ids']
        prompt_length = len(prompt_ids)
        
        # Create label ids by copying input_ids
        label_ids = input_ids.copy()
        
        # Mask prompt part: positions before prompt_length → -100
        label_ids[:prompt_length] = [-100] * prompt_length
        
        labels.append(label_ids)
    
    model_inputs['labels'] = labels
    return model_inputs

# Task 2: Custom Data Collation

Batching + Padding

In [15]:
class QADataCollator:
    """
    Custom data collator for QA fine-tuning with proper padding
    """
    def __init__(self, tokenizer, max_length=256):
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __call__(self, features):
        """
        TODO: Implement proper batching with padding
        1. Extract input_ids, attention_mask, labels from each feature
        2. Pad sequences to max_length
        3. Use pad_token_id for input_ids, 0 for attention_mask, -100 for labels
        4. Return tensors
        """
        # ✅ 1. Extract lists of input_ids, attention_mask, labels from features
        input_ids = [f['input_ids'] for f in features]
        attention_masks = [f['attention_mask'] for f in features]
        labels = [f['labels'] for f in features]

        # ✅ 2. Compute max_seq_length in this batch (capped at self.max_length)
        batch_max_len = min(max(len(ids) for ids in input_ids), self.max_length)

                # ✅ 3. Pad input_ids with tokenizer.pad_token_id
        padded_input_ids = [ids[:batch_max_len] + [self.tokenizer.pad_token_id] * (batch_max_len - len(ids)) for ids in input_ids]

        # ✅ 4. Pad attention_masks with 0
        padded_attention_masks = [mask[:batch_max_len] + [0] * (batch_max_len - len(mask)) for mask in attention_masks]

        # ✅ 5. Pad labels with -100 (for loss masking)
        padded_labels = [lbls[:batch_max_len] + [-100] * (batch_max_len - len(lbls)) for lbls in labels]

        # ✅ 6. Convert to tensors (efficient memory usage)
        batch = {
            'input_ids': torch.tensor(padded_input_ids, dtype=torch.long),
            'attention_mask': torch.tensor(padded_attention_masks, dtype=torch.long),
            'labels': torch.tensor(padded_labels, dtype=torch.long),
        }

        return batch


# Task 3: LoRA Con guration and Training

In [33]:
!pip -q install wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
import wandb
wandb.login(key="")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hivaabolhadi (hivaabolhadizadeh) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [16]:
# TODO: Apply preprocessing to datasets
tok_train_ds = train_ds.map(preprocess_function, batched=True, remove_columns=train_ds.column_names)
tok_eval_ds = eval_ds.map(preprocess_function, batched=True, remove_columns=eval_ds.column_names)

print(f"Training examples: {len(tok_train_ds)}")
print(f"Eval examples: {len(tok_eval_ds)}")

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Training examples: 2000
Eval examples: 200


In [17]:
# TODO: Setup LoRA configuration and model
from peft import LoraConfig, TaskType, get_peft_model

# TODO: Freeze base model parameters
for param in model.parameters():
    param.requires_grad = False

In [21]:
print(model.__class__)

<class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>


In [37]:
# TODO: Create LoRA configuration
lora_config = LoraConfig(
    # YOUR CODE HERE: Set r, lora_alpha, lora_dropout, target_modules, etc.
    task_type=TaskType.CAUSAL_LM,           # Specifies the task type → Causal Language Modeling (Autoregressive generation)
    r=8,                                    # Low-rank dimension for LoRA adapters → can experiment with 4, 16, or 32
    lora_alpha=16,                          # Scaling factor for LoRA → typically set to 2 * r
    lora_dropout=0.1,                       # Dropout rate for regularization to prevent overfitting
    target_modules=["attn.c_attn", "attn.c_proj"],# Apply LoRA only to query and value projection layers → can add 'mlp' for wider adaptation
    bias="none"
)

In [30]:
# TODO: Create LoRA model
lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


In [31]:
# Verify LoRA setup
for name, param in lora_model.named_parameters():
    if param.requires_grad:
        print(f"Trainable: {name}")

lora_model.train()

Trainable: base_model.model.transformer.h.0.attn.c_attn.lora_A.default.weight
Trainable: base_model.model.transformer.h.0.attn.c_attn.lora_B.default.weight
Trainable: base_model.model.transformer.h.0.attn.c_proj.lora_A.default.weight
Trainable: base_model.model.transformer.h.0.attn.c_proj.lora_B.default.weight
Trainable: base_model.model.transformer.h.0.mlp.c_proj.lora_A.default.weight
Trainable: base_model.model.transformer.h.0.mlp.c_proj.lora_B.default.weight
Trainable: base_model.model.transformer.h.1.attn.c_attn.lora_A.default.weight
Trainable: base_model.model.transformer.h.1.attn.c_attn.lora_B.default.weight
Trainable: base_model.model.transformer.h.1.attn.c_proj.lora_A.default.weight
Trainable: base_model.model.transformer.h.1.attn.c_proj.lora_B.default.weight
Trainable: base_model.model.transformer.h.1.mlp.c_proj.lora_A.default.weight
Trainable: base_model.model.transformer.h.1.mlp.c_proj.lora_B.default.weight
Trainable: base_model.model.transformer.h.2.attn.c_attn.lora_A.defau

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
           

In [32]:
# TODO: Create data collator instance
data_collator = QADataCollator(tokenizer, max_length=256)

In [40]:
import wandb

# W&B
wandb.init(project="lora_squad_finetuning", name="r8_lr2e-4_attn")

In [45]:
wandb.config.update({
    "rank": lora_config.r,
    "lora_alpha": lora_config.lora_alpha,
    "lora_dropout": lora_config.lora_dropout,
    "target_modules": lora_config.target_modules
})

In [38]:
from transformers import TrainingArguments

In [46]:
training_args = TrainingArguments(
    output_dir="./lora_squad_output",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    report_to="wandb",  #  W&B
    fp16=True,  # GPU
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_total_limit=2,
    gradient_accumulation_steps=2,  #(pitfall 2: Memory Leaks)
)

In [47]:
wandb.config.update({
    "learning_rate": training_args.learning_rate,
    "num_train_epochs": training_args.num_train_epochs,
    "batch_size": training_args.per_device_train_batch_size
})

In [49]:
from transformers import Trainer

In [50]:
# TODO: Create trainer
trainer = Trainer(
    # YOUR CODE HERE: Set model, args, datasets, data_collator
    model=lora_model,
    args=training_args,
    train_dataset=tok_train_ds,
    eval_dataset=tok_eval_ds,
    data_collator=data_collator,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [51]:
# TODO: Train the model
print("Starting training...")
trainer.train()

Starting training...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,No log,0.329071
2,0.416000,0.310377
3,0.416000,0.309063


TrainOutput(global_step=189, training_loss=0.3189207914645079, metrics={'train_runtime': 186.1897, 'train_samples_per_second': 32.225, 'train_steps_per_second': 1.015, 'total_flos': 781013331836928.0, 'train_loss': 0.3189207914645079, 'epoch': 3.0})

In [52]:
# TODO: Evaluate the model
evaluation_result = trainer.evaluate()
print("Evaluation results:", evaluation_result)
wandb.log({"eval_loss": evaluation_result["eval_loss"]})

Evaluation results: {'eval_loss': 0.3090629279613495, 'eval_runtime': 3.9098, 'eval_samples_per_second': 51.153, 'eval_steps_per_second': 3.325, 'epoch': 3.0}


In [54]:
# TODO: Save the fine-tuned model
lora_model.save_pretrained("./lora_squad_output")
wandb.save("./lora_squad_output/*")

wandb: WARNING Symlinked 5 files into the W&B run directory, call wandb.save again to sync new files.


['/kaggle/working/wandb/run-20250709_164757-yor1lfpv/files/lora_squad_output/README.md',
 '/kaggle/working/wandb/run-20250709_164757-yor1lfpv/files/lora_squad_output/adapter_config.json',
 '/kaggle/working/wandb/run-20250709_164757-yor1lfpv/files/lora_squad_output/checkpoint-126',
 '/kaggle/working/wandb/run-20250709_164757-yor1lfpv/files/lora_squad_output/adapter_model.safetensors',
 '/kaggle/working/wandb/run-20250709_164757-yor1lfpv/files/lora_squad_output/checkpoint-189']

In [55]:
import os
print("Files in lora_squad_output:", os.listdir("./lora_squad_output"))

Files in lora_squad_output: ['adapter_config.json', 'adapter_model.safetensors', 'checkpoint-189', 'checkpoint-126', 'README.md']


# Task 4: Inference and Evaluation 

In [61]:
def generate_answer(model, tokenizer, context, question, max_new_tokens=50, decoding_strategy="greedy", **kwargs):
    """
    Generate answer for a given context and question
    
    TODO: Implement inference function
    1. Create prompt
    2. Tokenize input
    3. Generate with appropriate sampling parameters
    4. Decode and extract only the generated answer part
    """
    prompt = f"Context: {context}\nQuestion: {question}\nAnswer: "
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=256).to(device)
    
    if decoding_strategy == "greedy":
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            pad_token_id=tokenizer.pad_token_id
        )
    elif decoding_strategy == "top_k":
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_k=kwargs.get("top_k", 50),
            pad_token_id=tokenizer.pad_token_id
        )
    elif decoding_strategy == "nucleus":
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=kwargs.get("top_p", 0.9),
            pad_token_id=tokenizer.pad_token_id
        )
    elif decoding_strategy == "temperature":
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=kwargs.get("temperature", 1.0),
            pad_token_id=tokenizer.pad_token_id
        )
    
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = decoded_output.split("Answer: ")[-1].strip()
    return answer

In [59]:
# TODO: Load fine-tuned model for testing
from peft import AutoPeftModelForCausalLM
ft_model = AutoPeftModelForCausalLM.from_pretrained("./lora_squad_output")
ft_model.to(device)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
           

In [62]:
# TODO: Test on validation examples
print("\n=== Testing Fine-tuned Model ===")
for i, example in enumerate(vali_ds):
    if i >= 3:
        break
    context = example['context']
    question = example['question']
    ground_truth = example['answers']['text'][0] if example['answers']['text'] else ''
    predicted_answer = generate_answer(ft_model, tokenizer, context, question, decoding_strategy="greedy")
    print(f"\nExample {i+1}:")
    print(f"Context: {context[:100]}...")
    print(f"Question: {question}")
    print(f"Predicted Answer: {predicted_answer}")
    print(f"Ground Truth: {ground_truth}")


=== Testing Fine-tuned Model ===

Example 1:
Context: Super Bowl 50 was an American football game to determine the champion of the National Football Leagu...
Question: Which NFL team represented the AFC at Super Bowl 50?
Predicted Answer: The Denver Broncos.
Ground Truth: Denver Broncos

Example 2:
Context: Super Bowl 50 was an American football game to determine the champion of the National Football Leagu...
Question: Which NFL team represented the NFC at Super Bowl 50?
Predicted Answer: The Denver Broncos.
Ground Truth: Carolina Panthers

Example 3:
Context: Super Bowl 50 was an American football game to determine the champion of the National Football Leagu...
Question: Where did Super Bowl 50 take place?
Predicted Answer: The Super Bowl 50 was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold
Ground Truth: Santa Clara, California


In [67]:
!pip -q install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [68]:
# TODO: Compute SQuAD metrics
predictions = []
references = []

In [69]:
for example in vali_ds:
    context = example['context']
    question = example['question']
    ground_truth = example['answers']['text'][0] if example['answers']['text'] else ''
    predicted_answer = generate_answer(ft_model, tokenizer, context, question, decoding_strategy="greedy")
    predictions.append({"id": example['id'], "prediction_text": predicted_answer})
    references.append({"id": example['id'], "answers": example['answers']})

try:
    squad_metric = evaluate.load("squad")  
    results = squad_metric.compute(predictions=predictions, references=references)
    print(f"\nSQuAD Metrics: {results}")
    wandb.log({"squad_exact_match": results["exact_match"], "squad_f1": results["f1"]})
except Exception as e:
    print(f"Could not load SQuAD metric: {e}")
    print("Install with: pip install evaluate")


SQuAD Metrics: {'exact_match': 40.0, 'f1': 49.77777777777778}


----------------------------------------------------------------------------------------------------------------

In [70]:
print("\n=== Testing Baseline Model ===")
baseline_model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)
baseline_predictions = []
for example in vali_ds:
    context = example['context']
    question = example['question']
    predicted_answer = generate_answer(baseline_model, tokenizer, context, question, decoding_strategy="greedy")
    baseline_predictions.append({"id": example['id'], "prediction_text": predicted_answer})

try:
    baseline_results = squad_metric.compute(predictions=baseline_predictions, references=references)
    print(f"\nBaseline SQuAD Metrics: {baseline_results}")
    wandb.log({"baseline_exact_match": baseline_results["exact_match"], "baseline_f1": baseline_results["f1"]})
except Exception as e:
    print(f"Could not compute baseline SQuAD metrics: {e}")


=== Testing Baseline Model ===

Baseline SQuAD Metrics: {'exact_match': 0.0, 'f1': 16.984206664864654}


In [71]:
print("\n=== Failure Case Analysis ===")
for i, (pred, ref) in enumerate(zip(predictions, references)):
    if pred["prediction_text"] != ref["answers"]["text"][0]:
        print(f"\nFailure Case {i+1}:")
        print(f"Question: {vali_ds[i]['question']}")
        print(f"Predicted: {pred['prediction_text']}")
        print(f"Ground Truth: {ref['answers']['text'][0]}")
        print(f"Context: {vali_ds[i]['context'][:100]}...")


=== Failure Case Analysis ===

Failure Case 1:
Question: Which NFL team represented the AFC at Super Bowl 50?
Predicted: The Denver Broncos.
Ground Truth: Denver Broncos
Context: Super Bowl 50 was an American football game to determine the champion of the National Football Leagu...

Failure Case 2:
Question: Which NFL team represented the NFC at Super Bowl 50?
Predicted: The Denver Broncos.
Ground Truth: Carolina Panthers
Context: Super Bowl 50 was an American football game to determine the champion of the National Football Leagu...

Failure Case 3:
Question: Where did Super Bowl 50 take place?
Predicted: The Super Bowl 50 was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold
Ground Truth: Santa Clara, California
Context: Super Bowl 50 was an American football game to determine the champion of the National Football Leagu...

Failure Ca

##  Generation Strategy Analysis

In [72]:
decoding_strategies = [
    {"strategy": "greedy"},
    {"strategy": "top_k", "top_k": 10},
    {"strategy": "top_k", "top_k": 25},
    {"strategy": "top_k", "top_k": 50},
    {"strategy": "nucleus", "top_p": 0.8},
    {"strategy": "nucleus", "top_p": 0.9},
    {"strategy": "nucleus", "top_p": 0.95},
    {"strategy": "temperature", "temperature": 0.7},
    {"strategy": "temperature", "temperature": 1.0},
    {"strategy": "temperature", "temperature": 1.3},
]

In [73]:
print("\n=== Decoding Strategy Analysis ===")
for strategy in decoding_strategies:
    strategy_name = strategy["strategy"]
    strategy_params = {k: v for k, v in strategy.items() if k != "strategy"}
    predictions = []
    for example in vali_ds:
        context = example['context']
        question = example['question']
        predicted_answer = generate_answer(ft_model, tokenizer, context, question, 
                                         decoding_strategy=strategy_name, **strategy_params)
        predictions.append({"id": example['id'], "prediction_text": predicted_answer})
    
    try:
        results = squad_metric.compute(predictions=predictions, references=references)
        print(f"\nStrategy: {strategy_name} {strategy_params}")
        print(f"SQuAD Metrics: {results}")
        wandb.log({f"{strategy_name}_{str(strategy_params)}_exact_match": results["exact_match"],
                   f"{strategy_name}_{str(strategy_params)}_f1": results["f1"]})
    except Exception as e:
        print(f"Could not compute SQuAD metrics for {strategy_name}: {e}")


=== Decoding Strategy Analysis ===

Strategy: greedy {}
SQuAD Metrics: {'exact_match': 40.0, 'f1': 49.77777777777778}

Strategy: top_k {'top_k': 10}
SQuAD Metrics: {'exact_match': 20.0, 'f1': 28.353658536585368}

Strategy: top_k {'top_k': 25}
SQuAD Metrics: {'exact_match': 0.0, 'f1': 8.791208791208792}

Strategy: top_k {'top_k': 50}
SQuAD Metrics: {'exact_match': 20.0, 'f1': 20.0}

Strategy: nucleus {'top_p': 0.8}
SQuAD Metrics: {'exact_match': 0.0, 'f1': 8.148148148148149}

Strategy: nucleus {'top_p': 0.9}
SQuAD Metrics: {'exact_match': 0.0, 'f1': 4.651162790697674}

Strategy: nucleus {'top_p': 0.95}
SQuAD Metrics: {'exact_match': 0.0, 'f1': 0.9523809523809523}

Strategy: temperature {'temperature': 0.7}
SQuAD Metrics: {'exact_match': 0.0, 'f1': 24.000000000000004}

Strategy: temperature {'temperature': 1.0}
SQuAD Metrics: {'exact_match': 0.0, 'f1': 12.352941176470589}

Strategy: temperature {'temperature': 1.3}
SQuAD Metrics: {'exact_match': 0.0, 'f1': 6.128048780487805}


In [74]:
def run_experiment(ranks=[4, 8, 16, 32, 64], lrs=[7e-5, 1e-4, 2e-4, 5e-4, 1e-3], 
                  target_modules=[["attn.c_attn", "attn.c_proj"], 
                                  ["attn.c_attn", "attn.c_proj", "mlp.c_fc", "mlp.c_proj"],
                                  ["q_proj", "v_proj", "attn.c_attn", "attn.c_proj"]]):
    # Initialize list to store results
    results = []
    
    # Check the number of validation samples
    print(f"Number of validation samples: {len(vali_ds)}")
    
    for r in ranks:
        for lr in lrs:
            for modules in target_modules:
                # Define unique run name for W&B
                run_name = f"r{r}_lr{lr}_{'_'.join(modules)}"
                print(f"\nRunning experiment: {run_name}")
                
                # Initialize W&B run
                wandb.init(project="lora_squad_finetuning", name=run_name)
                
                # Reset and load base model
                model = AutoModelForCausalLM.from_pretrained("gpt2", config=config).to(device)
                # Freeze all parameters to apply LoRA
                for param in model.parameters():
                    param.requires_grad = False
                
                # Configure LoRA
                lora_config = LoraConfig(
                    task_type=TaskType.CAUSAL_LM,
                    r=r,
                    lora_alpha=16,
                    lora_dropout=0.1,
                    target_modules=modules,
                    bias="none"
                )
                lora_model = get_peft_model(model, lora_config)
                
                # Log number of trainable parameters
                trainable_params = sum(p.numel() for p in lora_model.parameters() if p.requires_grad)
                wandb.log({"trainable_params": trainable_params})
                
                # Set up TrainingArguments
                training_args = TrainingArguments(
                    output_dir=f"./lora_{run_name}",
                    eval_strategy="epoch",
                    save_strategy="epoch",
                    learning_rate=lr,
                    per_device_train_batch_size=8,
                    per_device_eval_batch_size=8,
                    num_train_epochs=5,  # Increased epochs for better training
                    weight_decay=0.01,
                    logging_dir=f"./logs_{run_name}",
                    logging_steps=100,
                    report_to="wandb",
                    fp16=True,  # Enable mixed precision for efficiency
                    load_best_model_at_end=True,
                    metric_for_best_model="eval_loss",
                    save_total_limit=2,
                    gradient_accumulation_steps=4,  # Increased for stable gradients
                )
                
                # Log hyperparameters to W&B
                wandb.config.update({
                    "rank": r,
                    "learning_rate": lr,
                    "target_modules": modules,
                    "lora_alpha": 16,
                    "lora_dropout": 0.1,
                    "batch_size": training_args.per_device_train_batch_size,
                    "num_epochs": training_args.num_train_epochs
                })
                
                # Create Trainer instance
                trainer = Trainer(
                    model=lora_model,
                    args=training_args,
                    train_dataset=tok_train_ds,
                    eval_dataset=tok_eval_ds,
                    data_collator=data_collator,
                )
                
                # Train and evaluate the model
                trainer.train()
                eval_result = trainer.evaluate()
                
                # Log training and evaluation loss
                train_loss = trainer.state.log_history[-2].get("loss", float('inf'))
                eval_loss = eval_result["eval_loss"]
                wandb.log({
                    "train_loss": train_loss,
                    "eval_loss": eval_loss,
                    "training_steps": trainer.state.global_step,
                    "training_time": trainer.state.log_history[-1].get("train_runtime", 0)
                })
                
                # Compute SQuAD metrics for Greedy and Top-k strategies
                for strategy in [{"strategy": "greedy"}, {"strategy": "top_k", "top_k": 5}]:
                    strategy_name = strategy["strategy"]
                    strategy_params = {k: v for k, v in strategy.items() if k != "strategy"}
                    predictions = []
                    for example in vali_ds:
                        context = example['context']
                        question = example['question']
                        predicted_answer = generate_answer(lora_model, tokenizer, context, question, 
                                                         decoding_strategy=strategy_name, **strategy_params)
                        predictions.append({"id": example['id'], "prediction_text": predicted_answer})
                    
                    try:
                        squad_metric = evaluate.load("squad")
                        squad_results = squad_metric.compute(predictions=predictions, references=references)
                        wandb.log({
                            f"squad_exact_match_{strategy_name}_{str(strategy_params)}": squad_results["exact_match"],
                            f"squad_f1_{strategy_name}_{str(strategy_params)}": squad_results["f1"]
                        })
                    except Exception as e:
                        print(f"Could not compute SQuAD metrics for {strategy_name}: {e}")
                
                # Store results for this experiment
                results.append({
                    "rank": r,
                    "lr": lr,
                    "modules": modules,
                    "train_loss": train_loss,
                    "eval_loss": eval_loss,
                    "trainable_params": trainable_params,
                    "squad_exact_match": squad_results.get("exact_match", 0),
                    "squad_f1": squad_results.get("f1", 0)
                })
                
                # Save model and upload to W&B
                lora_model.save_pretrained(f"./lora_{run_name}")
                wandb.save(f"./lora_{run_name}/*")
                
                # Free up memory (pitfall 2: Memory Leaks)
                lora_model.cpu()
                del lora_model, trainer, model
                gc.collect()
                torch.cuda.empty_cache()
                
                # Finish W&B run
                wandb.finish()
    
    # Plot loss curves
    plt.figure(figsize=(12, 8))
    for result in results:
        plt.plot([1, 2, 3, 4, 5], [result["train_loss"]] * 5, label=f"Train r={result['rank']}, lr={result['lr']}, {result['modules']}")
        plt.plot([1, 2, 3, 4, 5], [result["eval_loss"]] * 5, '--', label=f"Eval r={result['rank']}, lr={result['lr']}, {result['modules']}")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss Curves")
    plt.legend()
    plt.show()
    wandb.log({"loss_plot": wandb.Image(plt)})
    
    # Plot parameter count vs. performance
    plt.figure(figsize=(8, 6))
    for result in results:
        plt.scatter(result["trainable_params"], result["eval_loss"], label=f"r={result['rank']}, lr={result['lr']}, {result['modules']}")
        plt.scatter(result["trainable_params"], result["squad_f1"], marker='x', label=f"F1 r={result['rank']}, lr={result['lr']}, {result['modules']}")
    plt.xlabel("Number of Trainable Parameters")
    plt.ylabel("Evaluation Loss / SQuAD F1")
    plt.title("Parameter Count vs. Performance")
    plt.legend()
    plt.show()
    wandb.log({"param_vs_performance_plot": wandb.Image(plt)})
    
    # Print computational cost analysis
    print("\nComputational Cost Analysis:")
    for result in results:
        print(f"r={result['rank']}, lr={result['lr']}, modules={result['modules']}: "
              f"Eval Loss={result['eval_loss']:.4f}, Params={result['trainable_params']}, "
              f"SQuAD F1={result['squad_f1']:.4f}, Exact Match={result['squad_exact_match']:.4f}")
    
    return results

In [ ]:
# Run the experiment
experiment_results = run_experiment()

Number of validation samples: 5

Running experiment: r4_lr7e-05_attn.c_attn_attn.c_proj


baseline_exact_match,▁
baseline_f1,▁
eval/loss,█▁▁▁
eval/runtime,▇▁█▁
eval/samples_per_second,▂█▁█
eval/steps_per_second,▂█▁█
eval_loss,▁
greedy_{}_exact_match,▁
greedy_{}_f1,▁
nucleus_{'top_p': 0.8}_exact_match,▁
nucleus_{'top_p': 0.8}_f1,▁


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,No log,2.306402
2,No log,1.258926


# Results